In [1]:
import pandas as pd

In [2]:
data_path = '../../data/2.2_temporally_updated_data.feather'
df_data = pd.read_feather(data_path)
df_data

,IsCanceled,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,...,ArrivalDateDayName,ArrivalDateMonthName,ADR_lag_1,ADR_lag_7,ADR_7d_avg,ADR_30d_avg,ADR_7d_std,ADR_30d_std,ADR_ewm_3,ADR_ewm_7
0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,...,Wednesday,July,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35499,0,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,...,Wednesday,July,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00
35500,0,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,...,Wednesday,July,80.00,NaN,NaN,NaN,NaN,NaN,40.00,20.00
35501,0,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,...,Wednesday,July,101.50,NaN,60.50,NaN,53.49,NaN,70.75,40.38
35502,0,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,...,Wednesday,July,101.50,NaN,94.33,NaN,12.41,NaN,86.12,55.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40039,0,2,0.0,0,HB,GBR,Online TA,TA/TO,0,0,...,Thursday,August,196.67,174.0,216.39,172.47,17.48,60.99,197.45,183.19
40040,0,2,1.0,0,HB,ITA,Online TA,TA/TO,0,0,...,Thursday,August,207.03,207.5,211.23,177.19,17.06,62.39,202.24,189.15
13794,1,2,0.0,0,HB,ESP,Online TA,TA/TO,0,0,...,Thursday,August,312.29,104.4,238.66,192.16,63.97,80.74,257.26,219.93
40038,0,2,0.0,0,HB,GBR,Offline TA/TO,TA/TO,0,0,...,Thursday,August,207.00,72.2,242.11,206.81,60.78,70.87,232.13,216.70


In [ ]:
def explode_reservations(df, start_date_col, end_date_col):
    """
    Explodes reservations in a DataFrame to individual days.

    Args:
        df (pandas.DataFrame): The DataFrame containing the reservations.
        start_date_col (str): The name of the column in `df` that represents the start date of each reservation.
        end_date_col (str): The name of the column in `df` that represents the end date of each reservation.

    Returns:
        pandas.DataFrame: The DataFrame with each reservation expanded to individual days.

    Raises:
        None

    Example:
        df = pd.DataFrame({'check_in': ['2022-01-01', '2022-02-01'],
                           'check_out': ['2022-01-04', '2022-02-03']})
        explode_reservations(df, 'check_in', 'check_out')

    """
    # Create a DataFrame with each guest's stay expanded to individual days
    # Generate a range of dates for each row and store it in a new column
    df['date_range'] = df.apply(lambda row: pd.date_range(start=row[start_date_col],
                                                          end=row[end_date_col] - pd.Timedelta(days=1)),
                                axis=1)

    # Explode the date_range column to create a row for each date in the range
    expanded_df = df.explode('date_range')

    # Drop the original check_in and check_out columns if they are no longer needed
    expanded_df = expanded_df.drop(columns=[start_date_col,
                                            end_date_col])

    # Rename the date_range column to date for clarity
    expanded_df = (expanded_df
                   .rename(columns={'date_range': 'Stay Date'})
                   .reset_index(drop=True)
                   .dropna(subset = 'Stay Date'))

    # Reset index if needed
    expanded_df = expanded_df

    return expanded_df